<a href="https://colab.research.google.com/github/Jandsy/ml_finance_imperial/blob/main/Coursework/CourseWork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **<center>Machine Learning and Finance </center>**


## <center> CourseWork 2024 - StatArb </center>



In this coursework, you will delve into and replicate selected elements of the research detailed in the paper **[End-to-End Policy Learning of a Statistical Arbitrage Autoencoder Architecture](https://arxiv.org/pdf/2402.08233.pdf)**. **However, we will not reproduce the entire study.**

## Overview

This study redefines Statistical Arbitrage (StatArb) by combining Autoencoder architectures and policy learning to generate trading strategies. Traditionally, StatArb involves finding the mean of a synthetic asset through classical or PCA-based methods before developing a mean reversion strategy. However, this paper proposes a data-driven approach using an Autoencoder trained on US stock returns, integrated into a neural network representing portfolio trading policies to output portfolio allocations directly.


## Coursework Goal

This coursework will replicate these results, providing hands-on experience in implementing and evaluating this innovative end-to-end policy learning Autoencoder within financial trading strategies.

## Outline

- [Data Preparation and Exploration](#Data-Preparation-and-Exploration)
- [Fama French Analysis](#Fama-French-Analysis)
- [PCA Analysis](#PCA-Analysis)
- [Ornstein Uhlenbeck](#Ornstein-Uhlenbeck)
- [Autoencoder Analysis](#Autoencoder-Analysis)



**Description:**
The Coursework is graded on a 100 point scale and is divided into five  parts. Below is the mark distribution for each question:

| **Problem**  | **Question**          | **Number of Marks** |
|--------------|-----------------------|---------------------|
| **Part A**   | Question 1            | 4                   |
|              | Question 2            | 1                   |
|              | Question 3            | 3                   |
|              | Question 4            | 3                   |
|              | Question 5            | 1                   |
|              | Question 6            | 3                   |
|**Part  B**    | Question 7           | 1                   |
|              | Question 8            | 5                   |
|              | Question 9            | 4                   |
|              | Question 10           | 5                   |
|              | Question 11           | 2                   |
|              | Question 12           | 3                   |
|**Part  C**    | Question 13          | 3                   |
|              | Question 14           | 1                   |
|              | Question 15           | 3                   |
|              | Question 16           | 2                   |
|              | Question 17           | 7                   |
|              | Question 18           | 6                   |
|              | Question 19           | 3                   |
|  **Part  D** | Question 20           | 3                   |
|              | Question 21           | 5                   |
|              | Question 22           | 2                   |
|  **Part  E** | Question 23           | 2                   |
|              | Question 24           | 1                   |
|              | Question 25           | 3                   |
|              | Question 26           | 10                  |
|              | Question 27           | 1                   |
|              | Question 28           | 3                   |
|              | Question 29           | 3                   |
|              | Question 30           | 7                   |




Please read the questions carefully and do your best. Good luck!

## Objectives



## 1. Data Preparation and Exploration
Collect, clean, and prepare US stock return data for analysis.

## 2. Fama French Analysis
Utilize Fama French Factors to isolate the idiosyncratic components of stock returns, differentiating them from market-wide effects. This analysis helps in understanding the unique characteristics of individual stocks relative to broader market trends.

## 3. PCA Analysis
Employ Principal Component Analysis (PCA) to identify hidden structures and reduce dimensionality in the data. This method helps in extracting significant patterns that might be obscured in high-dimensional datasets.

## 4. Ornstein-Uhlenbeck Process
Analyze mean-reverting behavior in stock prices using the Ornstein-Uhlenbeck process. This stochastic process is useful for modeling and forecasting based on the assumption that prices will revert to a long-term mean.

## 5. Building a Basic Autoencoder Model
Construct and train a standard Autoencoder to extract residual idiosyncratic risk.








# Data Preparation and Exploration


---
<font color=green>Q1: (4 Marks)</font>
<br><font color='green'>
Write a Python function that accepts a URL parameter and retrieves the NASDAQ-100 companies and their ticker symbols by scraping the relevant Wikipedia page using **[Requests](https://pypi.org/project/requests/)** and **[BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)**. Your function should return the data as a list of tuples, with each tuple containing the company name and its ticker symbol. Then, call your function with the appropriate Wikipedia page URL and print the data in a 'Company: Ticker' format.

</font>

---


In [105]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [106]:

def get_nasdaq100_companies(url): # create a function to get the data from the wikipedia page
    response = requests.get(url) 
    soup = BeautifulSoup(response.content, 'html.parser') # parse the html content
    table = soup.find('table', {'id': 'constituents'}) # find the table with the id 'constituents'
    rows = table.find_all('tr') # find all the rows in the table
    data = []
    for row in rows[1:]:
        cols = row.find_all('td')
        data.append((cols[0].text.strip(), cols[1].text.strip()))
    return data

url = 'https://en.wikipedia.org/wiki/NASDAQ-100' # url of the wikipedia page
data = get_nasdaq100_companies(url)

df = pd.DataFrame(data, columns=['Name', 'Symbol']) # create a pandas dataframe
df

,Name,Symbol
0,Adobe Inc.,ADBE
1,ADP,ADP
2,Airbnb,ABNB
3,Alphabet Inc. (Class A),GOOGL
4,Alphabet Inc. (Class C),GOOG
...,...,...
96,Walgreens Boots Alliance,WBA
97,Warner Bros. Discovery,WBD
98,"Workday, Inc.",WDAY
99,Xcel Energy,XEL


---
<font color=green>Q2: (1 Mark)</font>
<br><font color='green'>
Given a list of tuples representing NASDAQ-100 companies (where each tuple contains a company name and its ticker symbol), write a Python script to extract all ticker symbols into a separate list called `tickers_list`.
</font>
---


In [107]:
tickers_list = df['Symbol'].tolist() # get the tickers list
print(tickers_list)

['ADBE', 'ADP', 'ABNB', 'GOOGL', 'GOOG', 'AMZN', 'AMD', 'AEP', 'AMGN', 'ADI', 'ANSS', 'AAPL', 'AMAT', 'ASML', 'AZN', 'TEAM', 'ADSK', 'BKR', 'BIIB', 'BKNG', 'AVGO', 'CDNS', 'CDW', 'CHTR', 'CTAS', 'CSCO', 'CCEP', 'CTSH', 'CMCSA', 'CEG', 'CPRT', 'CSGP', 'COST', 'CRWD', 'CSX', 'DDOG', 'DXCM', 'FANG', 'DLTR', 'DASH', 'EA', 'EXC', 'FAST', 'FTNT', 'GEHC', 'GILD', 'GFS', 'HON', 'IDXX', 'ILMN', 'INTC', 'INTU', 'ISRG', 'KDP', 'KLAC', 'KHC', 'LRCX', 'LIN', 'LULU', 'MAR', 'MRVL', 'MELI', 'META', 'MCHP', 'MU', 'MSFT', 'MRNA', 'MDLZ', 'MDB', 'MNST', 'NFLX', 'NVDA', 'NXPI', 'ORLY', 'ODFL', 'ON', 'PCAR', 'PANW', 'PAYX', 'PYPL', 'PDD', 'PEP', 'QCOM', 'REGN', 'ROP', 'ROST', 'SIRI', 'SBUX', 'SNPS', 'TTWO', 'TMUS', 'TSLA', 'TXN', 'TTD', 'VRSK', 'VRTX', 'WBA', 'WBD', 'WDAY', 'XEL', 'ZS']


---
<font color=green>Q3: (3 Marks)</font>
<br><font color='green'>
Using **[yfinance](https://pypi.org/project/yfinance/)** library, write a Python script that accepts a list of stock ticker symbols. For each symbol, download the adjusted closing price data, store it in a dictionary with the ticker symbol as the key, and then convert the final dictionary into a Pandas DataFrame. Handle any errors encountered during data retrieval by printing a message indicating which symbol failed
</font>
---

In [108]:
# lets use yfinance to get the stock data write a Python script that accepts a list of stock ticker symbols. For each symbol, download the adjusted closing price data, store it in a dictionary with the ticker symbol as the key, and then convert the final dictionary into a Pandas DataFrame. Handle any errors encountered during data retrieval by printing a message indicating which symbol failed

import yfinance as yf

def get_stock_data(tickers_list):
    stock_data = {}
    for ticker in tickers_list:
        try:
            stock = yf.Ticker(ticker)
            stock_data[ticker] = stock.history('20y')['Close']
        except:
            print(f'Failed to get data for {ticker}')
    return stock_data

stock_data = get_stock_data(tickers_list)
df = pd.DataFrame(stock_data)
df


,ADBE,ADP,ABNB,GOOGL,GOOG,AMZN,AMD,AEP,AMGN,ADI,...,TSLA,TXN,TTD,VRSK,VRTX,WBA,WBD,WDAY,XEL,ZS
Date,,,,,,,,,,,,,,,,,,,,,
2004-06-01 00:00:00-04:00,22.438295,21.908672,NaN,NaN,NaN,2.511500,15.300000,14.187233,38.752434,30.617174,...,NaN,16.801424,NaN,NaN,9.270000,21.458759,NaN,NaN,7.947763,NaN
2004-06-02 00:00:00-04:00,22.638105,21.858988,NaN,NaN,NaN,2.517500,15.080000,14.264047,39.373455,30.383156,...,NaN,16.359621,NaN,NaN,9.140000,21.220934,NaN,NaN,7.900705,NaN
2004-06-03 00:00:00-04:00,22.428308,22.057709,NaN,NaN,NaN,2.470000,14.590000,14.047169,39.380581,29.700090,...,NaN,15.813862,NaN,NaN,8.800000,21.068493,NaN,NaN,7.858356,NaN
2004-06-04 00:00:00-04:00,22.688053,22.067650,NaN,NaN,NaN,2.547500,15.120000,13.979396,39.566193,30.136499,...,NaN,16.379114,NaN,NaN,9.250000,20.964832,NaN,NaN,7.872470,NaN
2004-06-07 00:00:00-04:00,23.657120,22.350826,NaN,NaN,NaN,2.588000,15.800000,14.331816,39.808876,31.534227,...,NaN,17.041822,NaN,NaN,9.190000,21.202650,NaN,NaN,7.971292,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-05-22 00:00:00-04:00,483.929993,254.020004,142.710007,176.380005,178.000000,183.130005,165.520004,91.480003,309.410004,240.160004,...,180.110001,202.539993,95.320000,253.639999,447.529999,16.469999,8.06,259.880005,55.520000,174.160004
2024-05-23 00:00:00-04:00,483.309998,251.490005,141.039993,173.550003,175.059998,181.050003,160.429993,89.279999,306.500000,234.559998,...,173.740005,197.289993,92.510002,249.250000,450.799988,15.950000,7.70,260.899994,54.049999,171.910004
2024-05-24 00:00:00-04:00,475.429993,248.899994,144.470001,174.990005,176.330002,180.750000,166.360001,88.970001,305.839996,232.509995,...,179.240005,199.179993,94.750000,251.580002,456.950012,16.030001,7.73,220.910004,53.720001,171.639999


---
<font color=green>Q4: (3 Marks)</font>
<br><font color='green'>
Write a Python script to analyze stock data stored in a dictionary `stock_data` (where each key is a stock ticker symbol, and each value is a Pandas Series of adjusted closing prices). The script should:
1. Convert the dictionary into a DataFrame.
2. Calculate the daily returns for each stock.
3. Identify columns (ticker symbols) with at least 2000 non-NaN values in their daily returns.
4. Create a new DataFrame that only includes these filtered ticker symbols.
5. Remove any remaining rows with NaN values in this new DataFrame.
</font>

---

In [109]:
'''
Write a Python script to analyze stock data stored in a dictionary `stock_data` (where each key is a stock ticker symbol, and each value is a Pandas Series of adjusted closing prices). The script should:
1. Convert the dictionary into a DataFrame.
2. Calculate the daily returns for each stock.
3. Identify columns (ticker symbols) with at least 2000 non-NaN values in their daily returns.
4. Create a new DataFrame that only includes these filtered ticker symbols.
5. Remove any remaining rows with NaN values in this new DataFrame.
'''
# 1. Convert the dictionary into a DataFrame.
df = pd.DataFrame(stock_data)
# 2. Calculate the daily returns for each stock.
df_daily_returns = df.pct_change()
# 3. Identify columns (ticker symbols) with at least 2000 non-NaN values in their daily returns.
valid_columns = df_daily_returns.columns[df_daily_returns.notnull().sum() >= 2000]
# 4. Create a new DataFrame that only includes these filtered ticker symbols.
df_filtered = df_daily_returns[valid_columns]
# 5. Remove any remaining rows with NaN values in this new DataFrame.
df_filtered = df_filtered.dropna()

print(df_filtered)


                               ADBE       ADP     GOOGL      GOOG      AMZN  \
Date                                                                          
2015-12-10 00:00:00-05:00 -0.006699  0.006004 -0.003292 -0.002861 -0.003715   
2015-12-11 00:00:00-05:00  0.027653 -0.024924 -0.012657 -0.014130 -0.033473   
2015-12-14 00:00:00-05:00  0.020127  0.015240  0.016151  0.012045  0.027744   
2015-12-15 00:00:00-05:00  0.008149  0.010284 -0.003213 -0.005844  0.001110   
2015-12-16 00:00:00-05:00  0.016380  0.008541  0.021708  0.019761  0.026008   
...                             ...       ...       ...       ...       ...   
2024-05-22 00:00:00-04:00  0.004317  0.007456 -0.008265 -0.008577 -0.000109   
2024-05-23 00:00:00-04:00 -0.001281 -0.009960 -0.016045 -0.016517 -0.011358   
2024-05-24 00:00:00-04:00 -0.016304 -0.010299  0.008297  0.007255 -0.001657   
2024-05-28 00:00:00-04:00  0.006310 -0.022499  0.008058  0.009584  0.007745   
2024-05-29 00:00:00-04:00 -0.001735 -0.013194 -0.002

---
<font color=green>Q5: (1 Mark)</font>
<br><font color='green'>
Download the dataset named `df_filtered_nasdaq_100` from the GitHub repository of the course.
</font>

---

In [110]:
# lets use github link to get the data
nasdaq100 = 'https://raw.githubusercontent.com/Jandsy/ml_finance_imperial/main/Coursework/df_filtered_nasdaq_100.csv'
df_filtered_nasdaq_100 = pd.read_csv(nasdaq100, index_col=0)
print(df_filtered_nasdaq_100)

                ADBE       ADP     GOOGL      GOOG      AMZN       AMD  \
Date                                                                     
2015-12-10 -0.006699  0.006004 -0.003292 -0.002861 -0.003715  0.042553   
2015-12-11  0.027653 -0.024924 -0.012657 -0.014130 -0.033473 -0.036735   
2015-12-14  0.020127  0.015241  0.016151  0.012045  0.027744 -0.008475   
2015-12-15  0.008149  0.010283 -0.003213 -0.005844  0.001110  0.008547   
2015-12-16  0.016380  0.008541  0.021708  0.019761  0.026008  0.076271   
...              ...       ...       ...       ...       ...       ...   
2024-05-06  0.015241  0.003514  0.005142  0.004971  0.013372  0.034396   
2024-05-07 -0.002674  0.009805  0.018739  0.018548  0.000318 -0.008666   
2024-05-08 -0.008471 -0.008894 -0.010920 -0.010521 -0.004026 -0.005245   
2024-05-09 -0.011166  0.009097  0.003424  0.002454  0.007979 -0.008007   
2024-05-10 -0.000746  0.006975 -0.007708 -0.007518 -0.010660 -0.003084   

                 AEP      AMGN       

---
<font color=green>Q6: (3 Marks) </font>
<br><font color='green'>
Conduct an in-depth analysis of the `df_filtered_nasdaq_100` dataset from GitHub. Answer the following questions:
- Which stock had the best performance over the entire period?
- What is the average daily return of 'AAPL'?
- What is the worst daily return? Provide the stock name and the date it occurred.
</font>

---

In [111]:
'''
which stock has the best performance over the entire period
'''
 # first we calculate the sum of the daily returns for each stock
stock_returns = df_filtered_nasdaq_100.sum()
# then we get the index of the stock with the highest sum
best_stock = stock_returns.idxmax() 
print(best_stock) 

NVDA


In [112]:
'''
What is the average daily return of 'AAPL'?
'''
average_daily_return = df_filtered_nasdaq_100['AAPL'].mean()
print(f'The average daily return for AAPL is :', average_daily_return)

The average daily return for AAPL is : 0.0010849409515136207


In [114]:
'''
What is the worst daily return? Provide the stock name and the date it occurred.

'''
# first we get the worst daily return
worst_daily_return = df_filtered_nasdaq_100.min().min()
# then we get the date of the worst daily return
worst_daily_return_date = df_filtered_nasdaq_100.stack()[df_filtered_nasdaq_100.stack() == worst_daily_return].index[0] # stack method is used to convert the dataframe into a series and then we get the index of the worst daily return
stock = worst_daily_return_date[1]
stock_name = yf.Ticker(stock).info['longName']
print(f'The worst daily return is {worst_daily_return} and it occurred on {worst_daily_return_date[0]} for {stock} ({stock_name})')


HTTPError: 404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v6/finance/quoteSummary/FANG?modules=financialData&modules=quoteType&modules=defaultKeyStatistics&modules=assetProfile&modules=summaryDetail&ssl=true

# Fama French Analysis

The Fama-French five-factor model is an extension of the classic three-factor model used in finance to describe stock returns. It is designed to better capture the risk associated with stocks and explain differences in returns. This model includes the following factors:

1. **Market Risk (MKT)**: The excess return of the market over the risk-free rate. It captures the overall market's premium.
2. **Size (SMB, "Small Minus Big")**: The performance of small-cap stocks relative to large-cap stocks.
3. **Value (HML, "High Minus Low")**: The performance of stocks with high book-to-market values relative to those with low book-to-market values.
4. **Profitability (RMW, "Robust Minus Weak")**: The difference in returns between companies with robust (high) and weak (low) profitability.
5. **Investment (CMA, "Conservative Minus Aggressive")**: The difference in returns between companies that invest conservatively and those that invest aggressively.

## Additional Factor

6. **Momentum (MOM)**: This factor represents the tendency of stocks that have performed well in the past to continue performing well, and the reverse for stocks that have performed poorly.

### Mathematical Representation

The return of a stock $R_i^t$ at time $t$ can be modeled as follows :

$$
R_i^t - R_f^t = \alpha_i^t + \beta_{i,MKT}^t(R_M^t - R_f^t) + \beta_{i,SMB}^t \cdot SMB^t + \beta_{i,HML}^t \cdot HML^t + \beta_{i,RMW}^t \cdot RMW^t + \beta_{i,CMA}^t \cdot CMA^t + \beta_{i,MOM}^t \cdot MOM^t + \epsilon_i^t
$$

Where:
- $ R_i^t $ is the return of stock $i$ at time $t$
- $R_f^t $is the risk-free rate at time $t$
- $ R_M^t $ is the market return at time $t$
- $\alpha_i^t $ is the abnormal return or alpha of stock $ i $ at time $t$
- $\beta^t $ coefficients represent the sensitivity of the stock returns to each factor at time $t$
- $\epsilon_i^t $ is the error term or idiosyncratic risk unique to stock $ i $ at time $t$

This model is particularly useful for identifying which factors significantly impact stock returns and for constructing a diversified portfolio that is optimized for given risk preferences.




---
<font color=green>Q7: (1 Mark) </font>
<br><font color='green'>
Download the `fama_french_dataset` from the course's GitHub account.
</font>

---

In [22]:
fama_french_dataset = 'https://raw.githubusercontent.com/Jandsy/ml_finance_imperial/main/Coursework/fama_french_dataset.csv'
df_fama_french = pd.read_csv(fama_french_dataset, index_col=0)
print(df_fama_french)

            Mkt-RF   SMB   HML   RMW   CMA     RF   Mom
1963-07-01   -0.67  0.02 -0.35  0.03  0.13  0.012 -0.21
1963-07-02    0.79 -0.28  0.28 -0.08 -0.21  0.012  0.42
1963-07-03    0.63 -0.18 -0.10  0.13 -0.25  0.012  0.41
1963-07-05    0.40  0.09 -0.28  0.07 -0.30  0.012  0.07
1963-07-08   -0.63  0.07 -0.20 -0.27  0.06  0.012 -0.45
...            ...   ...   ...   ...   ...    ...   ...
2024-03-22   -0.23 -0.98 -0.53  0.29 -0.37  0.021  0.43
2024-03-25   -0.26 -0.10  0.88 -0.22 -0.17  0.021 -0.34
2024-03-26   -0.26  0.10 -0.13 -0.50  0.23  0.021  0.09
2024-03-27    0.88  1.29  0.91 -0.14  0.58  0.021 -1.34
2024-03-28    0.10  0.45  0.48 -0.07  0.09  0.021 -0.44

[15290 rows x 7 columns]


---
<font color=green>Q8: (5 Marks)</font>
<br><font color='green'>

Write a Python function called `get_sub_df_ticker(ticker, date, df_filtered, length_history)` that extracts a historical sub-dataframe for a given `ticker` from `df_filtered`. The function should use `length_history` to determine the number of trading days to include, ending at the specified `date`. Return the sub-dataframe for the specified `ticker`.
</font>

---


In [23]:

def get_sub_df_ticker(ticker, date, df_filtered, length_history):
    """
    Extract a subset of data for a specific ticker within a defined date range.

    Parameters:
    ticker (str): The ticker symbol to filter.
    date (str or pd.Timestamp): The end date for the data subset.
    df_filtered (pd.DataFrame): The DataFrame containing the stock data.
    length_history (int): The length of the history to look back in years.

    Returns:
    pd.DataFrame: The subset of the DataFrame containing the specified ticker's data.
    """
    # Convert the end date to a pandas Timestamp
    end_date = pd.to_datetime(date)
    
    # Ensure the DataFrame index is in datetime format
    if not pd.api.types.is_datetime64_any_dtype(df_filtered.index):
        df_filtered.index = pd.to_datetime(df_filtered.index)

    # Calculate the start date by subtracting the length of history from the end date
    start_date = end_date - pd.DateOffset(years=length_history)

    # Ensure the date range exists within the DataFrame index
    if start_date < df_filtered.index.min():
        start_date = df_filtered.index.min()
    if end_date > df_filtered.index.max():
        end_date = df_filtered.index.max()

    # Ensure the ticker exists in the DataFrame columns
    if ticker not in df_filtered.columns:
        raise ValueError(f"Ticker '{ticker}' not found in the DataFrame columns")

    # Extract the subset of data within the date range for the specified ticker
    sub_df = df_filtered.loc[start_date:end_date, [ticker]]

    return sub_df

In [24]:
'''
sub_df = get_sub_df_ticker('AAPL', '2021-01-01', df_filtered_nasdaq_100, 5)
print(sub_df)
'''

"\nsub_df = get_sub_df_ticker('AAPL', '2021-01-01', df_filtered_nasdaq_100, 5)\nprint(sub_df)\n"

---
<font color=green>Q9: (4 Marks)</font>
<br><font color='green'>
Create a Python function named `df_ticker_with_fama_french(ticker, date, df_filtered, length_history, fama_french_data)` that uses `get_sub_df_ticker` to extract historical data for a specific `ticker`. Incorporate the Fama-French factors from `fama_french_data` into the extracted sub-dataframe. Adjust the ticker's returns by subtracting the risk-free rate ('RF') and add other relevant Fama-French factors ('Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', and 'Mom'). Return the resulting sub-dataframe.
</font>

---

In [25]:

def df_ticker_with_fama_french(ticker, date, df_filtered, length_history, fama_french_data):
    """
    Integrate Fama-French data with the stock data for a specific ticker.

    Parameters:
    ticker (str): The ticker symbol to filter.
    date (str or pd.Timestamp): The end date for the data subset.
    df_filtered (pd.DataFrame): The DataFrame containing the stock data.
    length_history (int): The length of the history to look back in years.
    fama_french_data (pd.DataFrame): The DataFrame containing Fama-French factors.

    Returns:
    pd.DataFrame: The combined DataFrame with adjusted returns and Fama-French factors.
    """
    # Get the subset of the data for the specified ticker
    sub_df = get_sub_df_ticker(ticker, date, df_filtered, length_history)

    # Ensure Fama-French data index is in datetime format
    if not pd.api.types.is_datetime64_any_dtype(fama_french_data.index):
        fama_french_data.index = pd.to_datetime(fama_french_data.index)

    # Join the Fama-French data with the stock data
    sub_df = sub_df.join(fama_french_data, how='inner')

    # Adjusting the returns by subtracting the risk-free rate
    sub_df['Adj_Return'] = sub_df[ticker] - sub_df['RF']

    # Add other relevant Fama-French factors
    factors = ['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'Mom']
    for factor in factors:
        if factor in fama_french_data.columns:
            sub_df[factor] = sub_df[factor]

    return sub_df


'''
# test the function
sub_df_fama_french = df_ticker_with_fama_french('AAPL', '2021-01-01', df_filtered_nasdaq_100, 5, df_fama_french)
print(sub_df_fama_french)
'''


"\n# test the function\nsub_df_fama_french = df_ticker_with_fama_french('AAPL', '2021-01-01', df_filtered_nasdaq_100, 5, df_fama_french)\nprint(sub_df_fama_french)\n"

---
<font color=green>Q10: (5 Marks) </font>
<br><font color='green'>
Write a Python function named `extract_beta_fama_french` to perform a rolling regression analysis for a given stock at a specific time point using the Fama-French model. The function should accept the following parameters:

- `ticker`: A string indicating the stock symbol.
- `date`: A string specifying the date for the analysis.
- `length_history`: An integer representing the number of days of historical data to include.
- `df_filtered`: A pandas DataFrame (assumed to be derived from question 5) containing filtered stock data.
- `fama_french_data`: A pandas DataFrame (assumed to be from question 7) that includes Fama-French factors.

Utilize the `statsmodels.api` library to conduct the regression.
</font>

---

In [26]:
import statsmodels.api as sm
def extract_beta_fama_french(ticker, date, length_history, df_filtered, fama_french_data):
    """
    Perform a rolling regression analysis for a given stock at a specific time point using the Fama-French model.

    Parameters:
    ticker (str): A string indicating the stock symbol.
    date (str): A string specifying the date for the analysis.
    length_history (int): An integer representing the number of days of historical data to include.
    df_filtered (pd.DataFrame): A pandas DataFrame containing filtered stock data.
    fama_french_data (pd.DataFrame): A pandas DataFrame that includes Fama-French factors.

    Returns:
    Regression results of the Fama-French model.
    """
    # Integrate Fama-French data with the stock data
    df_combined = df_ticker_with_fama_french(ticker, date, df_filtered, length_history, fama_french_data)

    # Define the independent variables (Fama-French factors)
    X = df_combined[['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'Mom']]
    X = sm.add_constant(X)  # Add a constant term to the model

    # Define the dependent variable (adjusted returns)
    y = df_combined['Adj_Return']

    # Perform the regression
    model = sm.OLS(y, X).fit()

    return model


---
<font color=green>Q11: (2 Marks) </font>
<br><font color='green'>
Apply the `extract_beta_fama_french` function to the stock symbol 'AAPL' for the date '2024-03-28', using a historical data length of 252 days. Ensure that the `df_filtered` and `fama_french_data` DataFrames are correctly prepared and available in your environment before executing this function. The parameters for the function call are set as follows:

- **Ticker**: 'AAPL'
- **Date**: '2024-03-28'
- **Length of History**: 252 days
</font>

---



In [27]:
model = extract_beta_fama_french('AAPL', '2024-03-28', 252, df_filtered_nasdaq_100, df_fama_french)
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:             Adj_Return   R-squared:                       0.573
Model:                            OLS   Adj. R-squared:                  0.572
Method:                 Least Squares   F-statistic:                     466.2
Date:                Thu, 30 May 2024   Prob (F-statistic):               0.00
Time:                        08:10:54   Log-Likelihood:                 6133.5
No. Observations:                2088   AIC:                        -1.225e+04
Df Residuals:                    2081   BIC:                        -1.221e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0059      0.000    -20.877      0.0

---
<font color=green>Q12: (2 Marks)</font>
<br><font color='green'>
Once the `extract_beta_fama_french` function has been applied to 'AAPL' with the specified parameters, the next step is to analyze the regression summary to identify which Fama-French factor explains the most variance in 'AAPL' returns during the specified period.

Follow these steps to perform the analysis:

1. **Review the Summary**: Examine the regression output, focusing on the coefficients and their statistical significance (p-values).
2. **Identify Key Factor**: Determine which factor has the highest absolute coefficient value and is statistically significant (typically p < 0.05). This factor can be considered as having the strongest influence on 'AAPL' returns for the period.

</font>

---

In [28]:
'''
1. **Review the Summary**: Examine the regression output, focusing on the coefficients and their statistical significance (p-values).
'''
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:             Adj_Return   R-squared:                       0.573
Model:                            OLS   Adj. R-squared:                  0.572
Method:                 Least Squares   F-statistic:                     466.2
Date:                Thu, 30 May 2024   Prob (F-statistic):               0.00
Time:                        08:10:57   Log-Likelihood:                 6133.5
No. Observations:                2088   AIC:                        -1.225e+04
Df Residuals:                    2081   BIC:                        -1.221e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0059      0.000    -20.877      0.0

**Write your answers here:**

In [29]:
'''
2. **Identify Key Factor**: Determine which factor has the highest absolute coefficient value and is statistically significant (typically p < 0.05). This factor can be considered as having the strongest influence on 'AAPL' returns for the period.
'''
# Get the coefficients and p-values for the factors
coefficients = model.params[1:]
p_values = model.pvalues[1:]

# Filter the factors with p-values less than 0.05
significant_factors = p_values[p_values < 0.05]

# Identify the factor with the highest absolute coefficient value
key_factor = significant_factors.abs().idxmax()

print(f'The key factor that explains the most variance in AAPL returns is: {key_factor}')

The key factor that explains the most variance in AAPL returns is: SMB


# PCA Analysis


In literature, another method exists for extracting residuals for each stock, utilizing the PCA approach to identify hidden factors in the data. Let's describe this method.

The return of a stock $R_i^t$ at time $t$ can be modeled as follows :

$$
R_i^t  = \sum_{j=1}^m\beta_{i,j}^t F_j^t  + \epsilon_i^t
$$

Where:
- $ R_i^t $ is the return of stock $i$ at time $t$
- $m$ is the number of factors selected from PCA
-  $ F_j^t $ is the $j$-th hidden factor constructed from PCA at time $t$
- $\beta_{i,j}^t $ are the coefficients representing the sensitivity of the stock returns to each hidden factor.
- $\epsilon_i^t $  is the residual term for stock $i$ at time $t$, representing the portion of the return not explained by the PCA factors.

### Representation of Stock Return Data

Consider the return data for $N$ stocks over $T$ periods, represented by the matrix $R$ of size $T \times N$:

$$
R = \left[
\begin{array}{cccc}
R_1^T & R_2^T & \cdots & R_N^T \\
R_1^{T-1} & R_2^{T-1} & \cdots & R_N^{T-1} \\
\vdots & \vdots & \ddots & \vdots \\
R_1^1 & R_2^1 & \cdots & R_N^1 \\
\end{array}
\right]
$$

Each element $R_i^k$ of the matrix represents the return of stock $i$ at time $k$ and is defined as:

$$
R_i^k = \frac{S_{i,k} - S_{i, k-1}}{S_{i, k-1}}, \quad k=1,\cdots, T, \quad i=1,\cdots,N
$$

where $S_{i,k}$ denotes the adjusted close price of stock $i$ at time $k$.

### Standardization of Returns

To adjust for varying volatilities across stocks, we standardize the returns as follows:

$$
Z_i^t = \frac{R_i^t - \mu_i}{\sigma_i}
$$

where $\mu_i$ and $\sigma_i$ are the mean and standard deviation of returns for stock $i$ over the period $[t-T, t]$, respectively.

### Empirical Correlation Matrix

The empirical correlation matrix $C$ is computed from the standardized returns:

$$
C = \frac{1}{T-1} Z^T Z
$$

where $Z^T$ is the transpose of matrix $Z$.

### Singular Value Decomposition (SVD)

We apply Singular Value Decomposition to the correlation matrix $C$:

$$
C = U \Sigma V^T
$$

Here, $U$ and $V$ are orthogonal matrices representing the left and right singular vectors, respectively, and $\Sigma$ is a diagonal matrix containing the singular values, which are the square roots of the eigenvalues.

### Construction of Hidden Factors

For each of the top $m$ components, we construct the selected hidden factors as follows:

$$
F_j^t = \sum_{i=1}^N \frac{\lambda_{i,j}}{\sigma_i} R_i^t
$$

where $\lambda_{i,j}$ is the $i$-th component of the $j$-th eigenvector (ranked by eigenvalue magnitude).


---
<font color=green>Q13 (3 Marks):

For the specified period from March 29, 2023 ('2023-03-29'), to March 28, 2024 ('2024-03-28'), generate the matrix $Z$ by standardizing the stock returns using the DataFrame `df_filtered_new`
</font>

---


In [76]:
## Insert your code here
start_date = '2023-03-29'
end_date = '2024-03-28'
df_filtered_new = df_filtered.loc[start_date: end_date]

In [77]:
print(f"The DataFrame's First Date is {df_filtered_new.index[0]}, and Last Date is {df_filtered_new.index[-1]}")

The DataFrame's First Date is 2023-03-29 00:00:00-04:00, and Last Date is 2024-03-28 00:00:00-04:00


In [78]:
Z = (df_filtered_new - df_filtered_new.mean()) / df_filtered_new.std()
Z.head()

,ADBE,ADP,GOOGL,GOOG,AMZN,AMD,AEP,AMGN,ADI,ANSS,...,TTWO,TMUS,TSLA,TXN,VRSK,VRTX,WBA,WBD,WDAY,XEL
Date,,,,,,,,,,,,,,,,,,,,,
2023-03-29 00:00:00-04:00,0.658925,2.189516,0.106285,0.207821,1.503570,0.444433,1.020469,0.727394,1.860839,0.356414,...,0.517876,0.615262,0.814986,1.344644,1.105776,0.127319,0.497111,0.429736,2.277783,1.269715
2023-03-30 00:00:00-04:00,0.273051,-0.221306,-0.389329,-0.434766,0.787034,0.526996,0.277297,0.076722,1.630406,0.936497,...,-0.109884,0.439952,0.233493,1.187062,0.239468,-0.525980,0.691236,0.446100,0.389915,0.430698
2023-03-31 00:00:00-04:00,0.360570,1.136314,1.540732,1.439604,0.531735,-0.056439,0.475356,0.008638,0.935425,1.044069,...,1.337544,0.117646,2.058867,0.640231,0.325258,0.538943,-0.008807,0.565842,1.611597,0.597545
2023-04-03 00:00:00-04:00,-0.713053,-2.259597,0.252735,0.407399,-0.591892,-0.600163,-0.086831,0.759726,-0.328378,-0.555499,...,-0.377687,1.191784,-2.030038,-0.684867,-0.214460,0.183344,1.205797,-0.547975,-0.634285,0.121519
2023-04-04 00:00:00-04:00,0.560730,-1.137425,0.099652,0.013877,0.658632,-0.342217,0.223115,0.872407,-0.399698,-0.127907,...,1.434972,-0.347680,-0.377655,-1.394519,-0.538698,-0.487155,0.553147,0.755053,-0.537767,1.011205


---
<font color=green>Q14: (1 Mark) </font>
<br><font color='green'>
Download the `Z_matrix` matrix from the course's GitHub account.
</font>

---

In [80]:
## Insert your code here
Z_matrix_github_path = r"https://raw.githubusercontent.com/Jandsy/ml_finance_imperial/main/Coursework/Z_matrix.csv"
Z_matrix = pd.read_csv(Z_matrix_github_path, index_col=0)
Z_matrix.head()

,ADBE,ADP,GOOGL,GOOG,AMZN,AMD,AEP,AMGN,ADI,ANSS,...,TTWO,TMUS,TSLA,TXN,VRSK,VRTX,WBA,WBD,WDAY,XEL
Date,,,,,,,,,,,,,,,,,,,,,
2023-03-29,0.658925,2.189521,0.106285,0.207821,1.503570,0.444433,1.020467,0.727398,1.860839,0.356414,...,0.517876,0.615242,0.814986,1.344650,1.105776,0.127319,0.497115,0.429736,2.277783,1.269711
2023-03-30,0.273051,-0.221306,-0.389329,-0.434766,0.787034,0.526996,0.277291,0.076716,1.630406,0.936497,...,-0.109884,0.439972,0.233493,1.187056,0.239468,-0.525980,0.691243,0.446100,0.389915,0.430715
2023-03-31,0.360570,1.136308,1.540732,1.439604,0.531735,-0.056439,0.475365,0.008639,0.935425,1.044069,...,1.337544,0.117646,2.058867,0.640237,0.325258,0.538943,-0.008816,0.565842,1.611597,0.597545
2023-04-03,-0.713053,-2.259591,0.252735,0.407399,-0.591892,-0.600163,-0.086824,0.759731,-0.328378,-0.555499,...,-0.377687,1.191774,-2.030038,-0.684867,-0.214460,0.183344,1.205798,-0.547975,-0.634285,0.121519
2023-04-04,0.560730,-1.137430,0.099652,0.013877,0.658632,-0.342217,0.223111,0.872403,-0.399698,-0.127907,...,1.434972,-0.347689,-0.377655,-1.394525,-0.538698,-0.487155,0.553153,0.755053,-0.537767,1.011197


---
<font color=green>Q15: (3 Marks) </font>
<br><font color='green'>
For the specified period from March 29, 2023 ('2023-03-29'), to March 28, 2024 ('2024-03-28'), compute the correlation matrix
$C$ using the matrix `Z_matrix`.
</font>

---

In [81]:
## Insert your code here
T = len(Z_matrix)
C_formula = (1/(T - 1)) * Z_matrix.T @ Z_matrix
C_formula

,ADBE,ADP,GOOGL,GOOG,AMZN,AMD,AEP,AMGN,ADI,ANSS,...,TTWO,TMUS,TSLA,TXN,VRSK,VRTX,WBA,WBD,WDAY,XEL
ADBE,1.000000,0.218513,0.397890,0.400601,0.463488,0.444032,-0.035967,0.198781,0.321991,0.387483,...,0.257931,0.102167,0.268863,0.326597,0.171580,0.164760,0.033955,0.099841,0.418110,0.019105
ADP,0.218513,1.000000,0.294213,0.298841,0.168206,0.045884,0.228457,0.214813,0.279607,0.238355,...,0.290311,0.113985,0.178128,0.297954,0.325258,0.176771,0.142369,0.243986,0.320836,0.164682
GOOGL,0.397890,0.294213,1.000000,0.997415,0.521199,0.371105,-0.006803,0.118938,0.222252,0.292286,...,0.238219,0.086673,0.267941,0.192188,0.178622,0.142447,0.052710,0.042072,0.289137,0.025701
GOOG,0.400601,0.298841,0.997415,1.000000,0.525626,0.371568,-0.004037,0.118296,0.223710,0.294542,...,0.242111,0.091456,0.268114,0.198044,0.180110,0.146190,0.060822,0.045516,0.293575,0.026392
AMZN,0.463488,0.168206,0.521199,0.525626,1.000000,0.463049,-0.010849,0.123745,0.290872,0.342042,...,0.222346,0.120301,0.303368,0.299500,0.144325,0.104962,0.017926,0.162937,0.403757,-0.058870
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
VRTX,0.164760,0.176771,0.142447,0.146190,0.104962,0.039540,0.239861,0.281759,0.110189,0.142121,...,0.180810,0.139184,0.144443,0.198258,0.251863,1.000000,0.159124,0.062726,0.101851,0.184369
WBA,0.033955,0.142369,0.052710,0.060822,0.017926,0.002629,0.309717,0.214701,0.208907,0.096813,...,0.115189,0.063538,0.168495,0.199627,0.038371,0.159124,1.000000,0.361533,0.010855,0.194839
WBD,0.099841,0.243986,0.042072,0.045516,0.162937,0.092733,0.325463,0.220342,0.310681,0.095919,...,0.129023,0.084315,0.282265,0.355450,0.002990,0.062726,0.361533,1.000000,0.160860,0.183837
WDAY,0.418110,0.320836,0.289137,0.293575,0.403757,0.334587,0.017659,0.068097,0.315426,0.382360,...,0.293949,0.142648,0.277744,0.346161,0.195588,0.101851,0.010855,0.160860,1.000000,-0.019310


---
<font color=green>Q16: (2 Marks) </font>
<br><font color='green'>
Refind the correlation matrix from the from March 29, 2023 ('2023-03-29'), to March 28, 2024 ('2024-03-28') using pandas correlation matrix method.
</font>

---

In [82]:
## Insert your code here
C = Z_matrix.corr()
C

,ADBE,ADP,GOOGL,GOOG,AMZN,AMD,AEP,AMGN,ADI,ANSS,...,TTWO,TMUS,TSLA,TXN,VRSK,VRTX,WBA,WBD,WDAY,XEL
ADBE,1.000000,0.218513,0.397890,0.400601,0.463488,0.444032,-0.035967,0.198781,0.321991,0.387483,...,0.257931,0.102167,0.268863,0.326597,0.171580,0.164760,0.033955,0.099841,0.418110,0.019105
ADP,0.218513,1.000000,0.294213,0.298841,0.168206,0.045884,0.228457,0.214813,0.279607,0.238355,...,0.290311,0.113985,0.178128,0.297954,0.325258,0.176771,0.142369,0.243986,0.320836,0.164682
GOOGL,0.397890,0.294213,1.000000,0.997415,0.521199,0.371105,-0.006803,0.118938,0.222252,0.292286,...,0.238219,0.086673,0.267941,0.192188,0.178622,0.142447,0.052710,0.042072,0.289137,0.025701
GOOG,0.400601,0.298841,0.997415,1.000000,0.525626,0.371568,-0.004037,0.118296,0.223710,0.294542,...,0.242111,0.091456,0.268114,0.198044,0.180110,0.146190,0.060822,0.045516,0.293575,0.026392
AMZN,0.463488,0.168206,0.521199,0.525626,1.000000,0.463049,-0.010849,0.123745,0.290872,0.342042,...,0.222346,0.120301,0.303368,0.299500,0.144325,0.104962,0.017926,0.162937,0.403757,-0.058870
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
VRTX,0.164760,0.176771,0.142447,0.146190,0.104962,0.039540,0.239861,0.281759,0.110189,0.142121,...,0.180810,0.139184,0.144443,0.198258,0.251863,1.000000,0.159124,0.062726,0.101851,0.184369
WBA,0.033955,0.142369,0.052710,0.060822,0.017926,0.002629,0.309717,0.214701,0.208907,0.096813,...,0.115189,0.063538,0.168495,0.199627,0.038371,0.159124,1.000000,0.361533,0.010855,0.194839
WBD,0.099841,0.243986,0.042072,0.045516,0.162937,0.092733,0.325463,0.220342,0.310681,0.095919,...,0.129023,0.084315,0.282265,0.355450,0.002990,0.062726,0.361533,1.000000,0.160860,0.183837
WDAY,0.418110,0.320836,0.289137,0.293575,0.403757,0.334587,0.017659,0.068097,0.315426,0.382360,...,0.293949,0.142648,0.277744,0.346161,0.195588,0.101851,0.010855,0.160860,1.000000,-0.019310


---
<font color=green>Q17: (7 Marks) </font>
<br><font color='green'>
Conduct Singular Value Decomposition on the correlation matrix $C$. Follow these steps:


1.   **Perform SVD**: Decompose the matrix $C$ into its singular values and vectors.
2.   **Rank Eigenvalues**: Sort the resulting singular values (often squared to compare to eigenvalues) in descending order.
3. **Select Components**: Extract the first 20 components based on the largest singular values.
4. **Variance Explained**: Print the variance explained by the first 20 Components and dimensions of differents matrix that you created.

</font>

---

In [83]:
## Insert your code here
# Performing SVD
U, S, Vt = np.linalg.svd(C)

# Ranking the Eigenvalues
singular_values_squared = S**2
sorted_indices = np.argsort(singular_values_squared)[::-1]
sorted_singular_values = singular_values_squared[sorted_indices]

# Select Top 20 Components
top_20_components = sorted_singular_values[:20]

# Variance Expained
variance_explained = top_20_components / np.sum(singular_values_squared)

# Printing results
print("Variance explained by the first 20 components:")
print(variance_explained)

# Printing dimensions of different matrices
print("\nDimensions of U matrix:", U.shape)
print("Dimensions of singular values vector (S):", S.shape)
print("Dimensions of Vt matrix:", Vt.shape)

Variance explained by the first 20 components:
[0.78617886 0.0864855  0.02462599 0.01223756 0.01065625 0.00673085
 0.00554871 0.00498553 0.00486848 0.00417346 0.00369662 0.00336624
 0.00315847 0.00281594 0.0025195  0.00228393 0.00220505 0.00211658
 0.00191477 0.00185626]

Dimensions of U matrix: (89, 89)
Dimensions of singular values vector (S): (89,)
Dimensions of Vt matrix: (89, 89)


---
<font color=green>Q18: (6 Marks) </font>
<br><font color='green'>
Extract the 20 hidden factors in a matrix F. Check that shape of F is $(252,20)$
</font>

</font>

---

In [90]:
## Insert your code here
top_20_Vt = Vt[:20, :].T
F = np.dot(df_filtered_new.values, top_20_Vt)
F.shape

(252, 20)

---
<font color=green>Q19: (3 Marks) </font>
<br><font color='green'>
Perform the Regression Analysis of 'AAPL' for the date '2024-03-28', using a historical data length of 252 days using previous $F$ Matrix. Compare the R-squared from the ones obtained at Q11.
</font>

</font>

---

In [98]:
aapl_returns = df_filtered_new['AAPL'].values

In [122]:
X = F
X = sm.add_constant(X)
y = aapl_returns
model = sm.OLS(y, X).fit()

In [126]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.566
Model:                            OLS   Adj. R-squared:                  0.528
Method:                 Least Squares   F-statistic:                     15.06
Date:                Thu, 30 May 2024   Prob (F-statistic):           1.28e-31
Time:                        19:25:08   Log-Likelihood:                 857.88
No. Observations:                 252   AIC:                            -1674.
Df Residuals:                     231   BIC:                            -1600.
Df Model:                          20                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0005      0.001     -0.884      0.3

# Ornstein Uhlenbeck

The Ornstein-Uhlenbeck process is defined by the following stochastic differential equation (SDE):

$$ dX_t = \theta (\mu - X_t) dt + \sigma dW_t $$

where:

- **$ X_t $**: The value of the process at time $ t $.
- **$ \mu $**: The long-term mean (equilibrium level) to which the process reverts.
- **$ \theta $**: The speed of reversion or the rate at which the process returns to the mean.
- **$ \sigma $**: The volatility (standard deviation), representing the magnitude of random fluctuations.
- **$ W_t $**: A Wiener process or Brownian motion that adds stochastic (random) noise.

This equation describes a process where the variable $ X_t $ moves towards the mean $ \mu $ at a rate determined by $ \theta $, with random noise added by $ \sigma dW_t $.

---
<font color=green>Q20: (3 Marks) </font>
<br><font color='green'>
In the context of mean reversion, which quantity should be modeled using an Ornstein-Uhlenbeck process?
</font>

---

**Write your answers here:**

---
<font color=green>Q21: (5 Marks) </font>
<br><font color='green'>
Explain how the parameters $ \theta $ and $ \sigma $ can be determined using the following equations. Also, detail the underlying assumptions:
$$ E[X] = \mu $$
$$ \text{Var}[X] = \frac{\sigma^2}{2\theta} $$
</font>

---

**Write your answers here:**

---
<font color=green>Q22: (2 Marks) </font>
<br><font color='green'>
Create a function named `extract_s_scores` which computes 's scores' for the last element in a list of floating-point numbers. This function calculates the scores using the following formula $ \text{s scores} = \frac{X_T - \mu}{\sigma} $ where `list_xi` represents a list containing a sequence of floating-point numbers $(X_0, \cdots, X_T)$.

</font>

---

In [ ]:
## Insert your code here

# Autoencoder Analysis

Autoencoders are neural networks used for unsupervised learning, particularly for dimensionality reduction and feature extraction. Training an autoencoder on the $Z_i$ matrix aims to identify hidden factors capturing the intrinsic structures in financial data.

### Architecture
- **Encoder**: Compresses input data into a smaller latent space representation.
  - *Input Layer*: Matches the number of features in the $Z_i$ matrix.
  - *Hidden Layers*: Compress data through progressively smaller layers.
  - *Latent Space*: Encodes the data into hidden factors.
- **Decoder**: Reconstructs input data from the latent space.
  - *Hidden Layers*: Gradually expand to the original dimension.
  - *Output Layer*: Matches the input layer to recreate the original matrix.

### Training
The autoencoder is trained by minimizing reconstruction loss, usually mean squared error (MSE), between the input $Z_i$ matrix and the decoder's output.

### Hidden Factors Extraction
After training, the encoder's latent space provides the most important underlying patterns in the stock returns.

---
<font color=green>Q23: (2 Marks) </font>
<br><font color='green'>
Modify the standardized returns matrix `Z_matrix` to reduce the influence of extreme outliers on model trainingby ensuring that all values in the matrix `Z_matrix` do not exceed 3 standard deviations from the mean. Specifically, cap these values at the interval $-3, 3]$. Store the adjusted values in a new matrix, `Z_hat`.
</font>

----

In [116]:
## Insert your code here
Z_hat = np.clip(Z_matrix, -3, 3)
Z_hat

,ADBE,ADP,GOOGL,GOOG,AMZN,AMD,AEP,AMGN,ADI,ANSS,...,TTWO,TMUS,TSLA,TXN,VRSK,VRTX,WBA,WBD,WDAY,XEL
Date,,,,,,,,,,,,,,,,,,,,,
2023-03-29,0.658925,2.189521,0.106285,0.207821,1.503570,0.444433,1.020467,0.727398,1.860839,0.356414,...,0.517876,0.615242,0.814986,1.344650,1.105776,0.127319,0.497115,0.429736,2.277783,1.269711
2023-03-30,0.273051,-0.221306,-0.389329,-0.434766,0.787034,0.526996,0.277291,0.076716,1.630406,0.936497,...,-0.109884,0.439972,0.233493,1.187056,0.239468,-0.525980,0.691243,0.446100,0.389915,0.430715
2023-03-31,0.360570,1.136308,1.540732,1.439604,0.531735,-0.056439,0.475365,0.008639,0.935425,1.044069,...,1.337544,0.117646,2.058867,0.640237,0.325258,0.538943,-0.008816,0.565842,1.611597,0.597545
2023-04-03,-0.713053,-2.259591,0.252735,0.407399,-0.591892,-0.600163,-0.086824,0.759731,-0.328378,-0.555499,...,-0.377687,1.191774,-2.030038,-0.684867,-0.214460,0.183344,1.205798,-0.547975,-0.634285,0.121519
2023-04-04,0.560730,-1.137430,0.099652,0.013877,0.658632,-0.342217,0.223111,0.872403,-0.399698,-0.127907,...,1.434972,-0.347689,-0.377655,-1.394525,-0.538698,-0.487155,0.553153,0.755053,-0.537767,1.011197
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-03-22,-1.146803,-0.516681,1.151431,1.085070,0.074915,0.081847,-0.150701,-0.278013,-0.555224,0.126867,...,0.046870,-0.246035,-0.386620,-0.054031,-0.476030,-0.091837,-0.421327,-0.946797,0.106149,-0.002851
2024-03-25,0.659349,-1.221008,-0.372489,-0.341074,0.109668,-0.292706,-0.084892,1.184710,-0.959285,-0.282029,...,-2.575803,0.240999,0.343241,-0.651297,-1.151639,-0.024341,0.166127,0.118796,-0.427841,0.321648
2024-03-26,-0.032708,0.234343,0.131651,0.109342,-0.556130,-0.244717,-0.377797,0.183364,-0.577464,0.317533,...,0.150958,-0.070195,0.960796,-1.177043,-0.384593,0.306386,-0.206326,-0.246683,0.237594,-0.878121


---
<font color=green>Q24: (1 Marks) </font>
<br><font color='green'>
Fetch the `Z_hat` data from GitHub, and we'll proceed with it now.
</font>



In [128]:
## Insert your code here
Z_hat_path = r"https://raw.githubusercontent.com/Jandsy/ml_finance_imperial/main/Coursework/Z_hat.csv"
Z_hat = pd.read_csv(Z_hat_path)
Z_hat

,Date,ADBE,ADP,GOOGL,GOOG,AMZN,AMD,AEP,AMGN,ADI,...,TTWO,TMUS,TSLA,TXN,VRSK,VRTX,WBA,WBD,WDAY,XEL
0,2023-03-29,0.658925,2.189521,0.106285,0.207821,1.503570,0.444433,1.020467,0.727398,1.860839,...,0.517876,0.615242,0.814986,1.344650,1.105776,0.127319,0.497115,0.429736,2.277783,1.269711
1,2023-03-30,0.273051,-0.221306,-0.389329,-0.434766,0.787034,0.526996,0.277291,0.076716,1.630406,...,-0.109884,0.439972,0.233493,1.187056,0.239468,-0.525980,0.691243,0.446100,0.389915,0.430715
2,2023-03-31,0.360570,1.136308,1.540732,1.439604,0.531735,-0.056439,0.475365,0.008639,0.935425,...,1.337544,0.117646,2.058867,0.640237,0.325258,0.538943,-0.008816,0.565842,1.611597,0.597545
3,2023-04-03,-0.713053,-2.259591,0.252735,0.407399,-0.591892,-0.600163,-0.086824,0.759731,-0.328378,...,-0.377687,1.191774,-2.030038,-0.684867,-0.214460,0.183344,1.205798,-0.547975,-0.634285,0.121519
4,2023-04-04,0.560730,-1.137430,0.099652,0.013877,0.658632,-0.342217,0.223111,0.872403,-0.399698,...,1.434972,-0.347689,-0.377655,-1.394525,-0.538698,-0.487155,0.553153,0.755053,-0.537767,1.011197
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,2024-03-22,-1.146803,-0.516681,1.151431,1.085070,0.074915,0.081847,-0.150701,-0.278013,-0.555224,...,0.046870,-0.246035,-0.386620,-0.054031,-0.476030,-0.091837,-0.421327,-0.946797,0.106149,-0.002851
248,2024-03-25,0.659349,-1.221008,-0.372489,-0.341074,0.109668,-0.292706,-0.084892,1.184710,-0.959285,...,-2.575803,0.240999,0.343241,-0.651297,-1.151639,-0.024341,0.166127,0.118796,-0.427841,0.321648
249,2024-03-26,-0.032708,0.234343,0.131651,0.109342,-0.556130,-0.244717,-0.377797,0.183364,-0.577464,...,0.150958,-0.070195,0.960796,-1.177043,-0.384593,0.306386,-0.206326,-0.246683,0.237594,-0.878121
250,2024-03-27,-0.363721,1.052056,-0.024166,-0.010591,0.315892,0.224881,2.192442,1.128111,1.411127,...,0.034693,0.474269,0.396878,1.991081,0.941800,-0.265792,1.179504,1.004419,-0.793726,2.193537


---
<font color=green>Q25: (3 Marks) </font>
<br><font color='green'>
Segment the standardized and capped returns matrix $\hat{Z}$ into two subsets for model training and testing. Precisly Allocate 70% of the data in $\hat{Z}$ to the training set $ \hat{Z}_{train} $ and Allocate the remaining 30% to the testing set $\hat{Z}_{test}$. Treat each stock within $\hat{Z}$ as an individual sample, by flattening temporal dependencies.
</font>



In [140]:
""" 
"Treat each stock within Z_hat as an individual sample, by flattening temporal dependencies" 
means that instead of considering the time series data for each stock as dependent on previous time points, 
you treat each stock's entire time series as a single sample. In other words, you ignore the temporal order 
and dependencies between the time points and consider the data for each stock independently.
"""

print(f"The Shape of Matrix Z_hat is: {Z_hat.shape}")
# Flatten Z_hat
flattened_z_hat = Z_hat.values.reshape(-1, Z_hat.shape[1])
print(f"The Shape of Flattend Z_hat is: {flattened_z_hat.shape}")

# Calculate split index
split_index = int(flattened_z_hat.shape[0] * 0.7)

# Split into training and testing sets
Z_train = flattened_z_hat[:split_index, :]
Z_test = flattened_z_hat[split_index:, :]

print(f"The Shape of Z_train is: {Z_train.shape}")
print(f"The Shape of Z_test is: {Z_test.shape}")


The Shape of Matrix Z_hat is: (252, 90)
The Shape of Flattend Z_hat is: (252, 90)
The Shape of Z_train is: (176, 90)
The Shape of Z_test is: (76, 90)


---
<font color=green>Q26: (10 Marks) </font>
<br><font color='green'>
Please create an autoencoder following the instructions provided in  **[End-to-End Policy Learning of a Statistical Arbitrage Autoencoder Architecture](https://arxiv.org/pdf/2402.08233.pdf)**, Use the model 'Variant 2' in Table 1.
</font>

---

In [ ]:
## Insert your code here

---
<font color=green>Q27 (1 Mark) :

Display all the parameters of the deep neural network.
</font>

---

In [ ]:
## Insert your code here

---
<font color=green>Q28: (3 Marks) </font>
<br><font color='green'>
Train your model using the Adam optimizer for 20 epochs with a batch size equal to 8 and validation split to 20%. Specify the loss function you've chosen.
</font>


In [ ]:
## Insert your code here

---
<font color=green>Q29: (3 Marks) </font>
<br><font color='green'>
Predict using the testing set and extract the residuals based on the methodology described in **[End-to-End Policy Learning of a Statistical Arbitrage Autoencoder Architecture](https://arxiv.org/pdf/2402.08233.pdf)**.
for 'NVDA' stock.
</font>

---

In [ ]:
## Insert your code here

<font color=green>Q30: (7 Marks) </font>
<br><font color='green'>
By reading carrefully the paper **[End-to-End Policy Learning of a Statistical Arbitrage Autoencoder Architecture](https://arxiv.org/pdf/2402.08233.pdf)**, answers the following question:
1. **Summarize the Key Actions**: Highlight the main experiments and methodologies employed by the authors in Section 5.
2. **Reproduction Steps**: Detail the necessary steps required to replicate the authors' approach based on the descriptions provided in the paper.
3. **Proposed Improvement**: Suggest one potential enhancement to the methodology that could potentially increase the effectiveness or efficiency of the model.



**Write your answers here:**








